# **1. Perkenalan Dataset**


Data ini merupakan data harian Indeks
Standar Pencemar Udara (ISPU) Di Kabupaten Gresik pada periode 11 Maret 2024
hingga 18 Agustus 2025, yang diambil dari situs resmi
https://aqicn.org/station/@474973/, yakni platform penyedia informasi real-time
mengenai Indeks Standar Pencemar Udara (ISPU) di Kabupaten Gresik


# **2. Import Library**

In [1]:
from google.colab import files
import pandas as pd
from google.colab import files

# **3. Memuat Dataset**

In [2]:
print("Upload file dataset dari perangkat Anda:")
uploaded = files.upload()

print("\nFile yang berhasil diupload:")
for filename in uploaded.keys():
    print(f"- {filename} ({len(uploaded[filename])} bytes)")

print(f"\nTotal {len(uploaded)} file berhasil diupload.")

# Ambil nama file yang benar dari dictionary uploaded
filename = list(uploaded.keys())[0]
df = pd.read_excel(filename)

print(f"\n5 baris pertama dari {filename}:")
print(df.head())

Upload file dataset dari perangkat Anda:


Saving DataISPU_raw.xlsx to DataISPU_raw.xlsx

File yang berhasil diupload:
- DataISPU_raw.xlsx (38153 bytes)

Total 1 file berhasil diupload.

5 baris pertama dari DataISPU_raw.xlsx:
   No    CO    NO2     O3  PM10  PM2.5    SO2  ISPU
0   1  0.28  12.98   8.15  4.30   6.29  23.70    23
1   2  0.28  14.61   8.32  0.48   7.06  23.40    23
2   3  0.28  18.19  10.25  2.98   9.80  22.00    32
3   4  0.28  12.85   8.00  9.90   7.57  21.80    24
4   5  0.28  14.75   8.00  7.79   6.03  21.08    20


# **4. Exploratory Data Analysis (EDA)**



In [3]:
print("=" * 50)
print("4. EXPLORATORY DATA ANALYSIS (EDA)")
print("=" * 50)

# 1. Informasi dasar dataset
print("\n1. INFORMASI DASAR DATASET")
print("-" * 30)
print(f"Jumlah baris: {df.shape[0]}")
print(f"Jumlah kolom: {df.shape[1]}")

# 2. Nama kolom
print("\n2. NAMA KOLOM")
print("-" * 30)
for i, col in enumerate(df.columns, 1):
    print(f"{i}. {col}")

# 3. Tipe data setiap kolom
print("\n3. TIPE DATA")
print("-" * 30)
print(df.dtypes)

# 4. Statistik deskriptif untuk kolom numerik
print("\n4. STATISTIK DESKRIPTIF (NUMERIK)")
print("-" * 30)
print(df.describe())

# 5. Cek missing values
print("\n5. MISSING VALUES")
print("-" * 30)
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage (%)': missing_percentage
})
print(missing_df[missing_df['Missing Values'] > 0])

if missing_df[missing_df['Missing Values'] > 0].empty:
    print("Tidak ada missing values dalam dataset.")

# 6. Cek duplikat
print("\n6. DUPLIKAT DATA")
print("-" * 30)
duplicates = df.duplicated().sum()
print(f"Jumlah baris duplikat: {duplicates}")

# 7. Distribusi data untuk beberapa kolom pertama (jika numerik)
print("\n7. DISTRIBUSI DATA (5 KOLOM PERTAMA NUMERIK)")
print("-" * 30)
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
if len(numeric_cols) > 0:
    for i, col in enumerate(numeric_cols[:5]):  # Hanya 5 kolom pertama
        print(f"\nKolom: {col}")
        print(f"  Mean: {df[col].mean():.2f}")
        print(f"  Median: {df[col].median():.2f}")
        print(f"  Std Dev: {df[col].std():.2f}")
        print(f"  Min: {df[col].min():.2f}")
        print(f"  Max: {df[col].max():.2f}")
else:
    print("Tidak ada kolom numerik dalam dataset.")

# 8. Unique values untuk kolom kategorikal
print("\n8. UNIQUE VALUES (KOLOM KATEGORIKAL)")
print("-" * 30)
categorical_cols = df.select_dtypes(include=['object']).columns
if len(categorical_cols) > 0:
    for i, col in enumerate(categorical_cols[:5]):  # Hanya 5 kolom pertama
        unique_count = df[col].nunique()
        print(f"\nKolom: {col}")
        print(f"  Jumlah unique values: {unique_count}")
        if unique_count <= 10:  # Tampilkan values jika tidak terlalu banyak
            print(f"  Values: {df[col].unique().tolist()}")
else:
    print("Tidak ada kolom kategorikal dalam dataset.")

print("\n" + "=" * 50)
print("EDA SELESAI - DATA MASIH RAW (TANPA PREPROCESSING)")
print("=" * 50)

4. EXPLORATORY DATA ANALYSIS (EDA)

1. INFORMASI DASAR DATASET
------------------------------
Jumlah baris: 526
Jumlah kolom: 8

2. NAMA KOLOM
------------------------------
1. No
2. CO
3. NO2
4. O3
5. PM10
6. PM2.5
7. SO2
8. ISPU

3. TIPE DATA
------------------------------
No         int64
CO       float64
NO2      float64
O3       float64
PM10     float64
PM2.5    float64
SO2      float64
ISPU       int64
dtype: object

4. STATISTIK DESKRIPTIF (NUMERIK)
------------------------------
              No          CO         NO2          O3        PM10       PM2.5  \
count  526.00000  526.000000  526.000000  526.000000  526.000000  526.000000   
mean   263.50000    0.092129   16.335589   13.389373   18.554259   14.556198   
std    151.98739    0.096765    9.466022   13.015074    8.467732    6.126203   
min      1.00000    0.010000    3.000000    2.000000    0.480000    3.510000   
25%    132.25000    0.050000    9.742500    7.750000   12.920000   10.765000   
50%    263.50000    0.060000

# **5. Data Preprocessing**

In [4]:
print("=" * 50)
print("DATA PREPROCESSING UNTUK RANDOM FOREST CLASSIFIER")
print("=" * 50)

# Simpan data asli
df_original = df.copy()

# 0. TENTUKAN TARGET TERLEBIH DAHULU
print("\n0. MENENTUKAN TARGET VARIABEL")
print("-" * 30)

# Hapus kolom 'No' karena tidak diperlukan
if 'No' in df.columns:
    df = df.drop('No', axis=1)
    print("Kolom 'No' telah dihapus (tidak diperlukan untuk ML)")

# Buat target kategorikal dari ISPU (jika belum ada)
if 'ISPU_Kategori' not in df.columns and 'ISPU' in df.columns:
    print("Membuat kategori ISPU dari nilai ISPU...")

    # Definisikan bins sesuai standar ISPU Indonesia
    bins = [0, 50, 100, 200, 300, float('inf')]
    labels = ['Baik', 'Sedang', 'Tidak Sehat', 'Sangat Tidak Sehat', 'Berbahaya']

    df['ISPU_Kategori'] = pd.cut(df['ISPU'], bins=bins, labels=labels, include_lowest=True)

    # Hapus kolom ISPU numerik jika ingin klasifikasi murni
    # df = df.drop('ISPU', axis=1)  # Opsional: hapus jika tidak ingin leak informasi

    print("Distribusi kategori ISPU:")
    print(df['ISPU_Kategori'].value_counts().sort_index())
    print(f"\nJumlah kategori: {df['ISPU_Kategori'].nunique()}")

    # Cek imbalance
    class_dist = df['ISPU_Kategori'].value_counts(normalize=True) * 100
    print("\nDistribusi kelas (%):")
    for category, percentage in class_dist.items():
        print(f"  {category}: {percentage:.1f}%")

# 1. HANDLING MISSING VALUES (Diperbaiki)
print("\n1. PENANGANAN MISSING VALUES")
print("-" * 30)

missing_before = df.isnull().sum()
if missing_before.sum() > 0:
    print(f"Total missing values: {missing_before.sum()}")
    print(missing_before[missing_before > 0])

    # PENTING: Pisahkan fitur dan target sebelum imputasi
    features = ['CO', 'NO2', 'O3', 'PM10', 'PM2.5', 'SO2']
    target = 'ISPU_Kategori'

    # Untuk data kecil (526 baris), lebih baik drop NA daripada imputasi
    df_cleaned = df.dropna()
    rows_dropped = df.shape[0] - df_cleaned.shape[0]

    print(f"\nMenggunakan dropna() karena data hanya {df.shape[0]} baris")
    print(f"Baris yang dihapus: {rows_dropped} ({rows_dropped/df.shape[0]*100:.1f}%)")

    df = df_cleaned
else:
    print("Tidak ada missing values")

# 2. HANDLING DUPLICATES
print("\n2. PENANGANAN DATA DUPLIKAT")
print("-" * 30)

duplicates = df.duplicated().sum()
if duplicates > 0:
    print(f"Menghapus {duplicates} data duplikat")
    df = df.drop_duplicates()
    print(f"Sisa {df.shape[0]} baris data")
else:
    print("Tidak ada data duplikat")

# 3. OUTLIER DETECTION (Hanya Informasi - Jangan dihapus!)
print("\n3. ANALISIS OUTLIER (Hanya Informasi)")
print("-" * 30)

# Random Forest robust terhadap outlier, jadi hanya untuk analisis
numeric_cols = ['CO', 'NO2', 'O3', 'PM10', 'PM2.5', 'SO2']
outlier_info = {}

for col in numeric_cols:
    if col in df.columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR

        outliers = df[(df[col] < lower) | (df[col] > upper)]
        perc = (len(outliers) / len(df)) * 100

        outlier_info[col] = {
            'count': len(outliers),
            'percentage': perc,
            'min': df[col].min(),
            'max': df[col].max()
        }

print("Outlier per kolom (batas IQR 1.5):")
for col, info in outlier_info.items():
    print(f"  {col:6}: {info['count']:3} outlier ({info['percentage']:5.1f}%)")

# 4. ENCODING TARGET (Jika diperlukan untuk modeling)
print("\n4. PREPARASI TARGET VARIABEL")
print("-" * 30)

if 'ISPU_Kategori' in df.columns:
    # Simpan mapping untuk interpretasi nanti
    category_mapping = {cat: idx for idx, cat in enumerate(df['ISPU_Kategori'].cat.categories)}
    print("Mapping kategori ke angka:")
    for category, idx in category_mapping.items():
        print(f"  {idx}: {category}")

    # Untuk analisis, kita bisa encode nanti di pipeline
    print("\nTarget siap untuk modeling dengan LabelEncoder")

# 5. PREPARASI FEATURE DAN TARGET
print("\n5. PREPARASI FEATURE DAN TARGET")
print("-" * 30)

# Definisikan fitur dan target
X = df[['CO', 'NO2', 'O3', 'PM10', 'PM2.5', 'SO2']]
y = df['ISPU_Kategori']

print(f"Jumlah fitur: {X.shape[1]}")
print(f"Jumlah sampel: {X.shape[0]}")
print(f"Target classes: {y.nunique()}")

# 6. TRAIN-TEST SPLIT (STRATIFIED)
print("\n6. TRAIN-TEST SPLIT (STRATIFIED)")
print("-" * 30)

from sklearn.model_selection import train_test_split

# Gunakan stratified split karena data tidak seimbang
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # PENTING: pertahankan distribusi kelas
)

print(f"Train set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test set:  {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

print("\nDistribusi kelas di Train set:")
print(y_train.value_counts().sort_index())
print("\nDistribusi kelas di Test set:")
print(y_test.value_counts().sort_index())

# 7. SCALING (OPSIONAL - Random Forest tidak butuh)
print("\n7. SCALING (OPSIONAL - Hanya untuk referensi)")
print("-" * 30)

print("Random Forest TIDAK MEMERLUKAN scaling karena berbasis tree")
print("Namun, scaling bisa membantu visualisasi atau algoritma lain")

# 8. FINAL SUMMARY
print("\n" + "=" * 50)
print("RINGKASAN AKHIR PREPROCESSING")
print("=" * 50)

print(f"\nDataset akhir:")
print(f"  Jumlah baris: {df.shape[0]}")
print(f"  Jumlah kolom: {df.shape[1]}")
print(f"  Fitur: {list(X.columns)}")
print(f"  Target: ISPU_Kategori")

print("\nStatistik dataset:")
print(f"  Missing values: {df.isnull().sum().sum()}")
print(f"  Duplicates: {df.duplicated().sum()}")

print("\nData siap untuk Random Forest Classifier!")
print("=" * 50)

# 8. FINAL SUMMARY
print("\n" + "=" * 50)
print("RINGKASAN AKHIR PREPROCESSING")
print("=" * 50)

# Save the cleaned data to a CSV file
output_file = "DataISPU_preprocessing.csv"
df.to_csv(output_file, index=False)
print(f"Data telah disimpan ke {output_file}")

# Provide a download link
files.download(output_file)

print(f"\nDataset akhir:")
print(f"  Jumlah baris: {df.shape[0]}")
print(f"  Jumlah kolom: {df.shape[1]}")
print(f"  Fitur: {list(X.columns)}")
print(f"  Target: ISPU_Kategori")

DATA PREPROCESSING UNTUK RANDOM FOREST CLASSIFIER

0. MENENTUKAN TARGET VARIABEL
------------------------------
Kolom 'No' telah dihapus (tidak diperlukan untuk ML)
Membuat kategori ISPU dari nilai ISPU...
Distribusi kategori ISPU:
ISPU_Kategori
Baik                  359
Sedang                167
Tidak Sehat             0
Sangat Tidak Sehat      0
Berbahaya               0
Name: count, dtype: int64

Jumlah kategori: 2

Distribusi kelas (%):
  Baik: 68.3%
  Sedang: 31.7%
  Tidak Sehat: 0.0%
  Sangat Tidak Sehat: 0.0%
  Berbahaya: 0.0%

1. PENANGANAN MISSING VALUES
------------------------------
Tidak ada missing values

2. PENANGANAN DATA DUPLIKAT
------------------------------
Menghapus 42 data duplikat
Sisa 484 baris data

3. ANALISIS OUTLIER (Hanya Informasi)
------------------------------
Outlier per kolom (batas IQR 1.5):
  CO    :  63 outlier ( 13.0%)
  NO2   :  29 outlier (  6.0%)
  O3    :  86 outlier ( 17.8%)
  PM10  :   9 outlier (  1.9%)
  PM2.5 :  18 outlier (  3.7%)
  SO2  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Dataset akhir:
  Jumlah baris: 484
  Jumlah kolom: 8
  Fitur: ['CO', 'NO2', 'O3', 'PM10', 'PM2.5', 'SO2']
  Target: ISPU_Kategori
